In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle # to serialize/unserialize python data objects


In [2]:
def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}
    
    from pandas import DataFrame
    from urllib.parse import urlencode
    import json
    
    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}

    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(DataFrame(d, columns=h))

In [3]:
params = {'PlayerID':'', # We'll fill this in after identifying the right player
          'PlayerPosition':'',
          'Season':'2017-18',
          'IsOnlyCurrentSeason' : "1",
          'ContextMeasure':'FGA',
          'DateFrom':'',
          'DateTo':'',
          'GameID':'',
          'GameSegment':'',
          'LastNGames':'0',
          'LeagueID':'00',
          'Location':'',
          'Month':'0',
          'OpponentTeamID':'0',
          'Outcome':'',
          'Period':'0',
          'Position':'',
          'RookieYear':'',
          'SeasonSegment':'',
          'SeasonType':'Playoffs',
          'TeamID':'0',
          'VsConference':'',
          'VsDivision':''}
 

In [5]:
with open('commonallplayers.json', 'r') as f:
    allplyrs = json.load(f)
#pickle.dump(allplyrs, open( "players.p", "wb" ) )
h = allplyrs['resultSets'][0]['headers']
d = allplyrs['resultSets'][0]['rowSet']

allplyrsdata= pd.DataFrame(d,columns= h) 
person_ids = allplyrsdata['PERSON_ID']
print(person_ids)

0       203518
1       203112
2       201167
3       201582
4       202332
5         2754
6      1628387
7         2546
8       201589
9      1628503
10      201600
11      202337
12     1627758
13     1627735
14      201563
15     1628444
16      204028
17      203505
18     1627762
19      202344
20      201166
21      202705
22     1626148
23      201628
24      203485
25      203900
26     1627852
27      202779
28      201596
29     1628492
        ...   
132    1627819
133     203104
134     203141
135    1628462
136       1891
137    1627755
138    1627756
139    1626173
140    1628476
141    1627866
142     204037
143    1627821
144    1627362
145       2561
146    1627855
147    1628510
148    1627785
149     203912
150    1628451
151       2863
152     202682
153    1628475
154     203966
155     203481
156     203489
157     201148
158     203923
159    1626202
160     201156
161    1627835
Name: PERSON_ID, Length: 162, dtype: int64


In [13]:
playerDict = {}
for i in person_ids.loc[:]:
    params["PlayerID"] = i
    plyr = get_nba_data('shotchartdetail', params, return_url=False)
    playerDict[i] = plyr
    fname = "{}.p"
    #pickle.dump(plyr, open( fname.format(i), "wb" ) )


In [15]:
pickle.dump(playerDict, open("allplayers.p","wb"))

In [17]:
playerDict = {}
params['SeasonType'] = "Regular Season"
for i in person_ids.loc[:]:
    params["PlayerID"] = i
    plyr = get_nba_data('shotchartdetail', params, return_url=False)
    playerDict[i] = plyr
    fname = "{}.p"


In [18]:
pickle.dump(playerDict, open("allplayersregular.p","wb"))